In [2]:
pwd

'/home/jovyan/critical_feature_extraction/ipynb'

In [3]:
cd ..

/home/jovyan/critical_feature_extraction


In [4]:
%run __init__.py
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Clean Data:
1. Download stock (Apple) TI's
2. Check data
3. Drop unnessesary columns

In [5]:
apple = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/AAPL_data.csv')

In [6]:
apple.head(1)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Date,Open,High,Low,Close,Volume,Name,...,RSI_12,STD_5,STD_20,SO%d_5,SO%d_20,SO%k,Trix_5,Trix_20,TSI_25_13,Target
0,61,61,61,2012-11-09,77.2,79.27,76.25,78.15,232476496,AAPL,...,0.071578,3.008766,4.566575,0.356641,0.372499,0.629139,-0.011429,-0.003303,-0.371455,0.779817


In [7]:
apple = apple.drop('Unnamed: 0', axis=1)
apple = apple.drop('Unnamed: 0.1', axis=1)
apple = apple.drop('Unnamed: 0.1.1', axis =1)
apple = apple.set_index(pd.to_datetime(apple.Date))
apple = apple.drop('Date', axis=1)
apple = apple.drop('Name', axis=1)
apple = apple.sort_index()

### Benchmarking with naive PCA and Linear Regression:
1. Prepare data for pipeline
2. Create a pipeline with:
    - Standardization
    - PCA
    - Linear Regression
3. Fit model
4. Produce scores
5. Visualize the Principal components

In [8]:
y = apple.Target
X = apple.drop('Target', axis=1)

In [9]:
#because this is time-series data we must manually split the data 
X_train = X.iloc[0:900, :]
y_train = y.iloc[0:900]
X_test = X.iloc[901:, :]
y_test = y.iloc[901:]

In [10]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('linreg', LinearRegression())
])

In [11]:
pipeline.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('linreg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [12]:
y_pred = pipeline.predict(X_test)

In [15]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Explain Variance: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 0.07
Explain Variance: -2.95
